# Kunden Match <a class="anchor" id="chapter1"></a>

In [1]:
import re
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### 0.) b.) Loading datasets <a class="anchor" id="section_3_1"></a>

In [2]:
file_path = 'Fuzzy Match.xlsx'

#Salesforce Sheet
df_salesforce = pd.read_excel(file_path, sheet_name = 'Salesforce')
#Steps Sheet
df_steps = pd.read_excel(file_path, sheet_name = 'Steps')
#df_steps.head()
#df_salesforce.head()


##check this at the end when putting in the IDs

In [3]:
# Number of entries in each DataFrame

#coutnumer of rows
entries_steps = 6593
entries_salesforce = 4469

# Calculate the difference in entries. 
difference = entries_steps - entries_salesforce
percentage_difference = (difference / entries_salesforce) * 100
print(f"Difference in entries: {difference}")
print(f"Percentage difference: {percentage_difference:.2f}%")

Difference in entries: 2124
Percentage difference: 47.53%


### 1.) Data manipulation for analysis<a class="anchor" id="section_3_1"></a>

### 1.) a.) Creating a concatenated "address" column for Steps<a class="anchor" id="section_3_1"></a>

In [4]:
# Check the data type of the 'Zip Code' column
zip_code_dtype = df_steps['Zip Code'].dtype
#'Zip Code'is a float that cannot be matched. For the string matching it will be transformed into a string

# Convert 'Zip Code', 'Street' and 'City' column in df_steps to string
df_steps['Zip Code'] = df_steps['Zip Code'].astype(str)
df_steps['Street'] = df_steps['Street'].astype(str)
df_steps['City'] = df_steps['City'].astype(str)

Creating a new column "Address" for both Salesforce and Steps
Create a concatenated address column to prepare the fuzzy string match. Therefore, the three columns Street, Zip Code, and City are concatenated in the new column 'Address'. Spaces are ignored - In this analysis, they don't add value

In [5]:
df_steps['Steps_Address'] = df_steps['Street'] + df_steps['Zip Code'] + df_steps['City']
# df_steps['Steps_Address'].head()

0                                       nannannan
1              Vredener Straße 4148703.0Stadtlohn
2        Kirchnerstraße 460311.0Frankfurt am Main
3               Stuttgarter Straße 2668782.0Brühl
4                   Eilbeker Weg 1622089.0Hamburg
                          ...                    
6588    Mannheimer Straße 22955543.0Bad Kreuznach
6589                   Lindenstr. 3917389.0Anklam
6590                      Worthweg 529693.0Ahlden
6591          Donnerburgweg 4038106.0Braunschweig
6592                    Liebenau 364252.0Liebenau
Name: Steps_Address, Length: 6593, dtype: object

### 1.) b.) Creating a concatenated "address" column for Salesforce<a class="anchor" id="section_3_1"></a>

In [6]:
#'Billing ZIP'is a float that cannot be matched. For the string matching it will be transformed into a string
df_salesforce['Billing Street'] = df_salesforce['Billing Street'].astype(str)
df_salesforce['Street Number'] = df_salesforce['Street Number'].astype(str)
df_salesforce['Billing ZIP'] = df_salesforce['Billing ZIP'].astype(str)
df_salesforce['Billing City'] = df_salesforce['Billing City'].astype(str)

In [7]:
df_salesforce['Salesforce_Address'] = df_salesforce['Billing Street'] + df_salesforce['Street Number'] + df_salesforce['Billing ZIP'] + df_salesforce['Billing City'] 
#df_salesforce['Salesforce_Address'].head()

### 1.) c.) Separating filled Steps ID<a class="anchor" id="section_3_1"></a>

-> Check how many Salesforce IDs are assigned a Steps ID. 

(1) This is relevant to exclude the IDs that were filled. "Einige wenige Kunden wurden bereits zwischen den Systemen gematcht, d.h. einer Salesforce-ID wurde eine Steps-ID zugewiesen."
(2) Fuzzywuzzy library expects non-null string inputs. Keeping in mind, if a null value is encountered, it could lead to errors.


In [8]:
#Which Step IDs aren't filled?
null_StepsID = df_salesforce['Steps ID'][df_salesforce['Steps ID'].isna()]

# To separate them for the string match, save the filled and non-filled ones as a new columns
df_salesforce['Filled_StepsID'] = df_salesforce['Steps ID'].where(df_salesforce['Steps ID'].notna())
df_salesforce['Manual_StepsID'] = df_salesforce['Steps ID'].where(df_salesforce['Steps ID'].isna())

# Validate the if the separation worked
print(df_salesforce['Filled_StepsID'].count()) #1067 Step IDs are already matched
print(df_salesforce['Manual_StepsID'].count())

1067
0


Match these back when completed the analysis

Finding the best match for each Salesforce address in Steps DataFrame

Snippet 1: Issue with this code: Figured there are missing and non-string (e.g. ZIP code) values. Elements used to create both Salesforce_Address or Steps_Address have non-string values and contain NaN values. There was a TypeError as process.extractOne() expected a strings or byte-like object.

### 2.) Fuzzy String Match<a class="anchor" id="section_3_1"></a>

### 2.) a.) Matching Salesforce and Steps concatenated addresses<a class="anchor" id="section_3_1"></a>

In [9]:
def find_best_match(salesforce_address, steps_address):
    best_match = process.extractOne(salesforce_address, steps_address, scorer=fuzz.token_sort_ratio)
    return best_match

# Apply the function to find matches
matches = df_salesforce['Salesforce_Address'].apply(find_best_match, steps_address = df_steps['Steps_Address'])

# Extract matched addresses and scores
df_salesforce['First_Best_Match_Address'] = matches.apply(lambda x: x[0] if x and x[1] >= 95 else None)
df_salesforce['First_Match_Score'] = matches.apply(lambda x: x[1] if x else None)

Best Match Address column df_salesforce DataFrame is empty. 
This indicates there weren't matches with a score of 95% or higher.

Mode for the first match

In [10]:
first_match_score_mode = df_salesforce['First_Match_Score'].mode()
print(f"Mode of Match Scores: {first_match_score_mode}")
#first_frequency_table = df_salesforce['First_Match_Score'].value_counts().sort_index()

# Display the frequency table
#print("Frequency table of Match Scores:")
#print(first_frequency_table)

Mode of Match Scores: 0    51
Name: First_Match_Score, dtype: int64


### 2.) b.) Excluding special characters<a class="anchor" id="section_3_1"></a>

In [11]:
# Remove special characters using RegEx to the 'Street' column in both df
df_steps['Steps_Address'] = df_steps['Steps_Address'].str.replace(r'[ \-./]', '', regex = True)
df_salesforce['Salesforce_Address'] = df_salesforce['Salesforce_Address'].str.replace(r'[ \-./]', '', regex = True)

# Display the DataFrame
#print(df_steps['Steps_Address'])
#print(df_salesforce['Salesforce_Address'])

2: Apply the Matching Function to Find Matches

In [12]:
# Apply the function to find matches
matches = df_salesforce['Salesforce_Address'].apply(find_best_match, steps_address = df_steps['Steps_Address'])

# Extract matched addresses and scores
df_salesforce['Second_Best_Match_Address'] = matches.apply(lambda x: x[0] if x and x[1] >= 95 else None)
df_salesforce['Second_Match_Score'] = matches.apply(lambda x: x[1] if x else None)


Removing the additional strings ("str", "straße", "strasse") from the addresses in both DataFrames and then applying the matching function.

In [14]:
# Removing the additional strings ("str", "straße", "strasse") from the addresses using Regex
names_to_remove = ['straße', 'str', 'strasse']
pattern = '|'.join(names_to_remove)

# Using Regex to remove names from columns
df_salesforce['Salesforce_Address'] = df_salesforce['Salesforce_Address'].str.replace(pattern, '', case=False, regex=True)
df_steps['Steps_Address'] = df_steps['Steps_Address'].str.replace(pattern, '', case=False, regex=True)

In [15]:
# Apply the function to find matches
matches = df_salesforce['Salesforce_Address'].apply(find_best_match, steps_address = df_steps['Steps_Address'])

# Extract matched addresses and scores
df_salesforce['Third_Best_Match_Address'] = matches.apply(lambda x: x[0] if x and x[1] >= 95 else None)
df_salesforce['Third_Match_Score'] = matches.apply(lambda x: x[1] if x else None)


Account Name (Salesforce) / Customer Name (Steps)
-> clean the addresses by removing specified characters and substrings from the columns "Account Name" in df_salesforce and "Customer Name" in df_steps

Apply the Function to Clean the Columns

In [17]:
# Regex pattern for the special characters to remove
pattern = r"[ \-\.\&\?\*\«\»\/\+\·\(\)\s®©!:,]"

# Remove the characters from 'Account Name' column in df_salesforce
df_salesforce['Cleaned_Account Name'] = df_salesforce['Account Name'].str.replace(pattern, '', regex=True)
#print(df_salesforce['Cleaned_Account Name'])

# Remove the characters from 'Customer Name' column in df_steps
df_steps['Cleaned_Customer Name'] = df_steps['Customer Name'].str.replace(pattern, '', regex=True)
#print(df_steps['Cleaned_Customer Name'])

Concatenate addresses with Account Name and Customer Name

In [18]:
# Concatenate 'Cleaned_Account Name' with 'Salesforce_Address'
df_salesforce['Salesforce_Address'] = df_salesforce['Cleaned_Account Name'] + df_salesforce['Salesforce_Address']
#print(df_salesforce['Salesforce_Address'].head(10))
# Concatenate 'Cleaned_Customer Name' with 'Steps_Address'
df_steps['Steps_Address'] = df_steps['Cleaned_Customer Name'] + df_steps['Steps_Address']
#print(df_steps['Steps_Address'].head())

In [19]:
# Apply the function to find matches
matches = df_salesforce['Salesforce_Address'].apply(find_best_match, steps_address = df_steps['Steps_Address'])

# Extract matched addresses and scores
df_salesforce['Fourth_Best_Match_Address'] = matches.apply(lambda x: x[0] if x and x[1] >= 95 else None)
df_salesforce['Fourth_Match_Score'] = matches.apply(lambda x: x[1] if x else None)


Removing legal forms and special characters from Account Name and Customer Name

In [21]:
legal_forms = [
    "kg", "e. k.", "e.K.", "gmbh", "GmbH \+ Co\. KG", "GmbH & Co\. KG",
    "e.Kfr\.", "e\.V\.", "AG", "ohg", "UG", "haftungsbeschränkt",
    "KG", "SE", "GbR", "GmbH&Co\.KG", "e\.U\.", "a\.G\.", "a\. G\.", 
    "m\.b\.H\.", "Inc\.", "B\.V\."
]

# Combine legal forms into a regex pattern
legal_forms_pattern = r"\b(?:{})\b".format('|'.join(re.escape(form) for form in legal_forms))

# Function to clean text by removing legal forms
def clean_text(text, pattern):
    if pd.isna(text):
        return text
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text.strip()

# Clean 'Salesforce_Address' in df_salesforce
df_salesforce['Cleaned_Salesforce_Address'] = df_salesforce['Salesforce_Address'].apply(clean_text, args=(legal_forms_pattern,))

# Clean 'Steps_Address' in df_steps
df_steps['Cleaned_Steps_Address'] = df_steps['Steps_Address'].apply(clean_text, args=(legal_forms_pattern,))

In [22]:
# Apply the function to find matches
matches = df_salesforce['Salesforce_Address'].apply(find_best_match, steps_address = df_steps['Steps_Address'])

# Extract matched addresses and scores
df_salesforce['Fifth_Best_Match_Address'] = matches.apply(lambda x: x[0] if x and x[1] >= 95 else None)
df_salesforce['Fifth_Match_Score'] = matches.apply(lambda x: x[1] if x else None)


Mapping 

In [46]:
# Create a mapping from address to Steps ID
address_to_id = dict(zip(df_salesforce['Fifth_Best_Match_Address'], df_steps['Steps ID']))

# Only keep the Steps ID if the Match Score is 90% or higher
df_salesforce['Filled_Steps_ID'] = df_salesforce.apply(
    lambda row: address_to_id[row['Fifth_Best_Match_Address']] if row['Fifth_Match_Score'] >= 90 else row['Steps ID'],
    axis=1
)


In [47]:
output_file = r'C:\Users\Katha\Documents\Job application documents\Data Analyst application\Case Studies\Rapid Data\df_salesforce.xlsx'

# Export the DataFrame to an Excel file
df_salesforce.to_excel(output_file, index=False, engine='openpyxl')

print(f"DataFrame has been exported to {output_file}")

DataFrame has been exported to C:\Users\Katha\Documents\Job application documents\Data Analyst application\Case Studies\Rapid Data\df_salesforce.xlsx
